This file contains the code of two stage processing of the data.
1. Improved DeepInsight method to generate gene expression data image with channel wise explanation (Original DeepInsight code can be accessed from the folloing link   https://alok-ai-lab.github.io/DeepInsight/)
2.Classification method of the generated image data by the vision Transformer
(Original Vision Transformer code can be accessed from the following link https://keras.io/examples/vision/image_classification_with_vision_transformer/)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE
from scipy.spatial import ConvexHull
from matplotlib import pyplot as plt
import inspect
#from pyDeepInsight import ImageTransformer, LogScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns


class ImageTransformer:
    """Transform features to an image matrix using dimensionality reduction
    This class takes in data normalized between 0 and 1 and converts it to a
    CNN compatible 'image' matrix
    """

    def __init__(self, feature_extractor='tsne', pixels=100,
                 random_state=None, n_jobs=-1):
        """Generate an ImageTransformer instance
        Args:
            feature_extractor: string of value ('tsne', 'pca', 'kpca') or a
                class instance with method `fit_transform` that returns a
                2-dimensional array of extracted features.
            pixels: int (square matrix) or tuple of ints (height, width) that
                defines the size of the image matrix.
            random_state: int or RandomState. Determines the random number
                generator, if present, of a string defined feature_extractor.
            n_jobs: The number of parallel jobs to run for a string defined
                feature_extractor.
        """
        self.random_state = random_state
        self.n_jobs = n_jobs

        if isinstance(feature_extractor, str):
            fe = feature_extractor.casefold()
            if fe == 'tsne'.casefold():
                fe = TSNE(
                    n_components=2, metric='cosine',
                    random_state=self.random_state)
            elif fe == 'pca'.casefold():
                fe = PCA(n_components=2,
                         random_state=self.random_state)
            elif fe == 'kpca'.casefold():
                fe = KernelPCA(
                    n_components=2, kernel='rbf',
                    random_state=self.random_state,
                    n_jobs=self.n_jobs)
            else:
                raise ValueError(("Feature extraction method '{}' not accepted"
                                  ).format(feature_extractor))
            self._fe = fe
        elif hasattr(feature_extractor, 'fit_transform') and \
                inspect.ismethod(feature_extractor.fit_transform):
            self._fe = feature_extractor
        else:
            raise TypeError('Parameter feature_extractor is not a '
                            'string nor has method "fit_transform"')

        if isinstance(pixels, int):
            pixels = (pixels, pixels)
        self._pixels = pixels
        self._xrot = None

    def fit(self, X, y=None, plot=False):
        """Train the image transformer from the training set (X)
        Args:
            X: {array-like, sparse matrix} of shape (n_samples, n_features)
            y: Ignored. Present for continuity with scikit-learn
            plot: boolean of whether to produce a scatter plot showing the
                feature reduction, hull points, and minimum bounding rectangle
        Returns:
            self: object
        """
        # perform dimensionality reduction
        x_new = self._fe.fit_transform(X.T)
        # get the convex hull for the points
        chvertices = ConvexHull(x_new).vertices
        hull_points = x_new[chvertices]
        # determine the minimum bounding rectangle
        mbr, mbr_rot = self._minimum_bounding_rectangle(hull_points)
        # rotate the matrix
        # save the rotated matrix in case user wants to change the pixel size
        self._xrot = np.dot(mbr_rot, x_new.T).T
        # determine feature coordinates based on pixel dimension
        self._calculate_coords()
        # plot rotation diagram if requested
        if plot is True:
            plt.scatter(x_new[:, 0], x_new[:, 1], s=1,
                        cmap=plt.cm.get_cmap("jet", 10), alpha=0.2)
            plt.fill(x_new[chvertices, 0], x_new[chvertices, 1],
                     edgecolor='r', fill=False)
            plt.fill(mbr[:, 0], mbr[:, 1], edgecolor='g', fill=False)
            plt.gca().set_aspect('equal', adjustable='box')
            plt.show()
        return self

    @property
    def pixels(self):
        """The image matrix dimensions
        Returns:
            tuple: the image matrix dimensions (height, width)
        """
        return self._pixels

    @pixels.setter
    def pixels(self, pixels):
        """Set the image matrix dimension
        Args:
            pixels: int or tuple with the dimensions (height, width)
            of the image matrix
        """
        if isinstance(pixels, int):
            pixels = (pixels, pixels)
        self._pixels = pixels
        # recalculate coordinates if already fit
        if hasattr(self, '_coords'):
            self._calculate_coords()

    def _calculate_coords(self):
        """Calculate the matrix coordinates of each feature based on the
        pixel dimensions.
        """
        ax0_coord = np.digitize(
            self._xrot[:, 0],
            bins=np.linspace(min(self._xrot[:, 0]), max(self._xrot[:, 0]),
                             self._pixels[0])
        ) - 1
        ax1_coord = np.digitize(
            self._xrot[:, 1],
            bins=np.linspace(min(self._xrot[:, 1]), max(self._xrot[:, 1]),
                             self._pixels[1])
        ) - 1
        self._coords = np.stack((ax0_coord, ax1_coord), axis=1)

    def transform(self, X, format='scalar', empty_value=0):
        """Transform the input matrix into image matrices
        Args:
            X: {array-like, sparse matrix} of shape (n_samples, n_features)
                where n_features matches the training set.
            format: The format of the image matrix to return. 'scalar' return a
                array of shape (M, N). 'rgb' returns an numpy.ndarray of shape
                (M, N, 3) that is compatible with PIL.
            empty_value: numeric value to fill elements where no features are
                mapped. Default = 0.
        Returns:
            A list of n_samples numpy matrices of dimensions set by
            the pixel parameter

        """

        img_coords = pd.DataFrame(np.vstack((
            self._coords.T,
            X
        )).T).groupby([0, 1], as_index=False).mean()


        img_matrices = []
        blank_mat = np.zeros(self._pixels)
        if empty_value != 0:
            blank_mat[:] = empty_value
        for z in range(2, img_coords.shape[1]):
            img_matrix = blank_mat.copy()
            img_matrix[img_coords[0].astype(int),
                       img_coords[1].astype(int)] = img_coords[z]
            img_matrices.append(img_matrix)


        if format=='rgb':
            img_matrices = np.array([self._mat_to_rgb(m) for m in img_matrices])
        elif format=='scalar':
            img_matrices = np.stack(img_matrices)
        else:
            raise ValueError(("'{}' not accepted for parameter 'format'")
                             .format(format))

        return img_matrices

    def fit_transform(self, X, **kwargs):
        """Train the image transformer from the training set (X) and return
        the transformed data.
        Args:
            X: {array-like, sparse matrix} of shape (n_samples, n_features)
        Returns:
            A list of n_samples numpy matrices of dimensions set by
            the pixel parameter
        """
        self.fit(X)
        return self.transform(X, **kwargs)

    def feature_density_matrix(self):
        """Generate image matrix with feature counts per pixel
        Returns:
            img_matrix (ndarray): matrix with feature counts per pixel
        """
        fdmat = np.zeros(self._pixels)
        np.add.at(fdmat, tuple(self._coords.T), 1)
        return fdmat

    def coords(self):
        """Get feature coordinates
        Returns:
            ndarray: the pixel coordinates for features
        """
        return self._coords.copy()

    @staticmethod
    def _minimum_bounding_rectangle(hull_points):
        """Find the smallest bounding rectangle for a set of points.
        Modified from JesseBuesking at https://stackoverflow.com/a/33619018
        Returns a set of points representing the corners of the bounding box.
        Args:
            hull_points : an nx2 matrix of hull coordinates
        Returns:
            (tuple): tuple containing
                coords (ndarray): coordinates of the corners of the rectangle
                rotmat (ndarray): rotation matrix to align edges of rectangle
                    to x and y
        """

        pi2 = np.pi / 2
        # calculate edge angles
        edges = hull_points[1:] - hull_points[:-1]
        angles = np.arctan2(edges[:, 1], edges[:, 0])
        angles = np.abs(np.mod(angles, pi2))
        angles = np.unique(angles)
        # find rotation matrices
        rotations = np.vstack([
            np.cos(angles),
            -np.sin(angles),
            np.sin(angles),
            np.cos(angles)]).T
        rotations = rotations.reshape((-1, 2, 2))
        # apply rotations to the hull
        rot_points = np.dot(rotations, hull_points.T)
        # find the bounding points
        min_x = np.nanmin(rot_points[:, 0], axis=1)
        max_x = np.nanmax(rot_points[:, 0], axis=1)
        min_y = np.nanmin(rot_points[:, 1], axis=1)
        max_y = np.nanmax(rot_points[:, 1], axis=1)
        # find the box with the best area
        areas = (max_x - min_x) * (max_y - min_y)
        best_idx = np.argmin(areas)
        # return the best box
        x1 = max_x[best_idx]
        x2 = min_x[best_idx]
        y1 = max_y[best_idx]
        y2 = min_y[best_idx]
        rotmat = rotations[best_idx]
        # generate coordinates
        coords = np.zeros((4, 2))
        coords[0] = np.dot([x1, y2], rotmat)
        coords[1] = np.dot([x2, y2], rotmat)
        coords[2] = np.dot([x2, y1], rotmat)
        coords[3] = np.dot([x1, y1], rotmat)

        return coords, rotmat

    @staticmethod
    def _mat_to_rgb(mat):
        """Convert image matrix to numpy rgb format
        Args:
            mat: {array-like} (M, N)
        Returns:
            An numpy.ndarry (M, N, 3) with orignal values repeated across
            RGB channels.
        """
        return np.repeat(mat[:, :, np.newaxis], 3, axis=2)


class LogScaler:
    """Log normalize and scale data
    Log normalization and scaling procedure as described as norm-2 in the
    DeepInsight paper supplementary information.
    """

    def __init__(self):
        self._min0 = None
        self._max = None
        pass

    def fit(self, X, y=None):
        self._min0 = X.min(axis=0)
        self._max = np.log(X + np.abs(self._min0) + 1).max()

    def fit_transform(self, X, y=None):
        self._min0 = X.min(axis=0)
        X_norm = np.log(X + np.abs(self._min0) + 1)
        self._max = X_norm.max()
        return X_norm / self._max

    def transform(self, X, y=None):
        X_norm = np.log(X + np.abs(self._min0) + 1).clip(0, None)
        return (X_norm / self._max).clip(0, 1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Read the data from drive
df = pd.read_csv('/content/drive/MyDrive/New_MLL.csv')
# Note: For your data, you need to use the link of your Google drive

df = df.drop(['Unnamed: 0'],axis=1) # this line is used in my data one can remove for their data
data= df.values
X = data[:,:-1]
YY = data[:,-1]
scaler = MinMaxScaler(feature_range=(0, 1))
X_scl = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_scl, YY, test_size=0.2, random_state=23, stratify=YY)
X_train.shape

In [ ]:
ln = LogScaler()
X_train_norm = ln.fit_transform(X_train)
X_test_norm = ln.transform(X_test)
X_norm = ln.transform(X_scl)
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, perplexity=64, metric='cosine',
            random_state=1701)

it = ImageTransformer(feature_extractor=tsne, pixels=64)

In [ ]:
plt.figure(figsize=(5, 5))
_ = it.fit(X_norm, plot=True)

In [ ]:
fdm = it.feature_density_matrix()
data_pix=fdm
data_pix.max()

In [ ]:
# image genrated by original deepInsight
plt.figure(figsize=(5, 5))
_ = it.fit(X_train_norm, plot=True)
px_sizes = [25, (25, 50), 50, 100]

fig, ax = plt.subplots(1, len(px_sizes), figsize=(25, 7))
for ix, px in enumerate(px_sizes):
    it.pixels = px
    fdm = it.feature_density_matrix()

    fdm[fdm == 0] = np.nan
    cax = sns.heatmap(fdm, cmap="viridis", linewidth=0.01,
                      linecolor="lightgrey", square=True,
                      ax=ax[ix], cbar=False)
    cax.set_title('Dim {} x {}'.format(*it.pixels))
    for _, spine in cax.spines.items():
        spine.set_visible(True)
    cax.xaxis.set_major_locator(ticker.MultipleLocator(5))
    cax.yaxis.set_major_locator(ticker.MultipleLocator(5))
plt.tight_layout()

it.pixels =64

In [ ]:
data_pix.sum()
aaa=it.coords().T
aaa.shape
a_i=pd.DataFrame(np.vstack((aaa,X_norm)).T)
ai=pd.DataFrame(np.vstack((aaa,X_norm)).T).groupby([0, 1], as_index=False)
ai.groups
ai


The following piece of the code is for Improved DeepInsight method with  channel wise expansion.

In [ ]:
aaa=it.coords().T
aai=pd.DataFrame(np.vstack((aaa,X_norm)).T).groupby([0, 1], as_index=False).agg(pd.Series.tolist)
aai

In [ ]:
pxm=64 # for image size of 64X64
emp_value=0

In [ ]:
# create diffrent channels data
def newsamp(dfn,s_n):
    split_df = pd.DataFrame(dfn[s_n].tolist())
    split_df
    split_df[(split_df.select_dtypes(include=['number']) != 0).any(axis=1)]
    avg_no=split_df.shape[1]
    ave_data = split_df.copy()
    ave_data['average'] = ave_data.mean(numeric_only=True, axis=1)
    row=ave_data.shape[0]
    clo=ave_data.shape[1]
    AA=np.array(ave_data)
    for i in range(row):
        for j in range(clo):
            if np.isnan(AA[i][j]):
                AA[i][j] = AA[i][avg_no]
    AA=np.delete(AA, avg_no, 1)
    dfn = dfn.drop(s_n, axis=1)
    df_new = pd.DataFrame(AA, columns = [i for i in range(2,AA.shape[1]+2)])
    new_df = pd.concat([dfn, df_new], axis=1)
    return new_df

In [ ]:
#create image of each channel
def image_mat(newdf):
    im_matrices = []
    blank_m = np.zeros((pxm,pxm))
    if emp_value !=0:
        blank_m[:]=emp_value
    for zz in range(2, newdf.shape[1]):
        im_matrix=blank_m.copy()
        im_matrix[newdf[0].astype(int),
              newdf[1].astype(int)] = newdf[zz]
        im_matrices.append(im_matrix)
    im_matrices = np.stack(im_matrices)
    return im_matrices

In [ ]:
#dataset of channel wise expantion images
list_image=[]
for i in range(2,aai.shape[1]):
    dfn=aai[[0,1,i]]
    #print(dfn)
    newdf=newsamp(dfn,i)
    img_15=image_mat(newdf)
    list_image.append(img_15)
gene_data=np.stack(list_image)

In [ ]:
# generate proper format for images
from numpy import moveaxis
import numpy as np
ch_image=[]
for i in range(gene_data.shape[0]):
    data = moveaxis(gene_data[i], 0, 2)
    ch_image.append(data)
img_data=np.stack(ch_image)#Final image data

In [ ]:
print(img_data.shape)
plt.imshow(img_data[0,:,:,0])

The following code is for the classification of data.

In [ ]:
pip install -U tensorflow-addons

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
#Divide data into train test data
Y = to_categorical(YY)
x_train, x_test, y_train, y_test = train_test_split(
    img_data, YY, test_size=0.2, random_state=23, stratify=YY)
#x_train.shape

In [ ]:
num_classes = len(np.unique(YY))
input_shape = (64, 64, img_data.shape[3])

Vision Transformer code can be read from following link https://keras.io/examples/vision/image_classification_with_vision_transformer/

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(4, 4))
image = x_train[np.random.choice(range(x_train.shape[0]))]
plt.imshow(image[:,:,0])
#plt.axis("off")

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    #augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)( inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.1)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.1)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [ ]:
learning_rate = 0.0001
weight_decay = 0.001
batch_size = 30
num_epochs = 10
image_size = 64  # We'll resize input images to this size
patch_size = 8 # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 8
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048,1024]  # Size of the dense layers of the final classifier

In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdaBelief(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(img_data, YY)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history


vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)

In [ ]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from imblearn.metrics import specificity_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
yhat_classes = vit_classifier.predict(img_data, verbose=0)
y_t = yhat_classes[:, 1]
x_c=np.argmax(yhat_classes,axis=1)
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(YY,x_c)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(YY, x_c,average='macro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(YY, x_c,average='macro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(YY, x_c,average='macro')
print('F1 score: %f' % f1)
# specificity= tn/(fp+tn)
specificity=specificity_score(YY, x_c, average='macro')
print('specificity: %f' % specificity)
# kappa
kappa = cohen_kappa_score(YY, x_c)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(Y, yhat_classes)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(YY, x_c)
print(matrix)
from mlxtend.plotting import plot_confusion_matrix
plt.rcParams.update({'font.size': 22})
fig1, ax = plot_confusion_matrix(conf_mat=matrix, figsize=(6, 6), )
plt.xlabel('Predictions Class', fontsize=18)
plt.ylabel('Actual Class', fontsize=18)
plt.title('confusion_matrix', fontsize=23)
plt.show()